In [6]:
#First we load the data and packages.
import pandas as pd
import numpy as np
from copy import deepcopy
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# the command below only needs to be run once
#import sys
# !{sys.executable} -m pip install xgboost
import xgboost as xgb
#from xgboost.xgbclassifier import XGBClassifier as xgb

In [7]:
# load train and test data, make dummies from genres and combine it with df.
df1 = pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv",index_col=0)
genres = df1.genres.str.get_dummies(sep = "|")

df = pd.concat([df1, genres], axis=1)
df = df.drop(['genres'], axis = 1)

In [30]:
#create features and target data
data = df.drop(['rating', 'title'], axis= 1)
target = df[['rating']]
df.head()

,userId,movieId,rating,title,year,(no genres listed),Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,Toy Story,1995,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,5,1,4.0,Toy Story,1995,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,7,1,4.5,Toy Story,1995,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,15,1,2.5,Toy Story,1995,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,17,1,4.5,Toy Story,1995,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#split df in training and test data
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state= 101)

In [23]:
#build a content-based model with XGBoost
model1 = xgb.XGBRegressor(objective='reg:squarederror')
model1.fit(X_train, y_train) 

pred1 = model1.predict(X_test)
rmse = np.sqrt(np.mean((pred1 - y_test.to_numpy())**2))
print(f'content-based rmse = {rmse}')


#matrix factorization

content-based rmse = 1.1594521885467903


In [33]:
display(pred1)
display(y_test)

array([3.6006124, 3.270881 , 2.9481487, ..., 4.2712545, 4.114047 ,
       3.1766407], dtype=float32)

,rating
78807,2.0
76710,3.0
70748,1.5
46156,3.0
80102,3.5
...,...
46058,3.0
56608,0.5
41606,5.0
742,3.0


In [26]:
 def recommend_similar_items(self,movie_title,n=5):
        movieId=df[df['title']==movie_title]['movieId'].unique()[0]
        movie_neighbours=self.model.get_neighbors(movieId,n)
        
        df_out=df[df.movieId.isin(movie_neighbours)].drop(["userId"],axis=1)
        df_out=df_out.groupby(['movieId','title',"year","genres"]).mean()
        df_out['average rating']=df_out['rating'].round(2)
        df_out=df_out.drop(['rating'],axis=1)
        df_out['number of ratings']=len(df[df.movieId.isin(movie_neighbours)]['rating'])
        
        return df_out 

array([3.6006124, 3.270881 , 2.9481487, ..., 4.2712545, 4.114047 ,
       3.1766407], dtype=float32)

In [27]:
#best_model = GridSearchCV(estimator=XGBRegressor(),
#                         param_grid=hyperparameters,
#                         n_jobs=-1,
#                         cv=10,
 #                        verbose=2)

# Fit the model
#best_model = best_model.fit(X_train,y_train)